In [1]:
#%pip install opencv-python deepface mysql-connector-python numpy scipy

In [2]:
import os
import time
import uuid
import cv2
import glob
import shutil
import json
import random
import numpy as np
from datetime import datetime
import mysql.connector
from deepface import DeepFace
from scipy.spatial.distance import cosine

# === Configuration ===

In [3]:
WATCH_FOLDER = "captured_faces"
KNOWN_FOLDER = "known_faces"
ARCHIVE_ROOT = "Process"
MATCH_THRESHOLD = 0.45  # distance threshold (lower == stricter)
BLUR_THRESHOLD = 100.0
MODEL_NAME = "ArcFace"  # DeepFace model to use for embeddings
EMBEDDING_SIZE = None  # will be inferred


# === MySQL Connection ===

In [4]:
db = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="emotion_detection"
)
cursor = db.cursor()

In [5]:
def ensure_tables_exist(conn):
    cur = conn.cursor()
    cur.execute("""
        CREATE TABLE IF NOT EXISTS face_embeddings (
            id INT AUTO_INCREMENT PRIMARY KEY,
            face_id VARCHAR(128) NOT NULL,
            embedding LONGTEXT NOT NULL,
            created_at DATETIME NOT NULL
        ) ENGINE=InnoDB;
    """)
    cur.execute("""
        CREATE TABLE IF NOT EXISTS unique_face_id (
            id INT AUTO_INCREMENT PRIMARY KEY,
            face_id VARCHAR(128) UNIQUE NOT NULL,
            embedding LONGTEXT,
            created_at DATETIME
        ) ENGINE=InnoDB;
    """)
    # Keep other tables as-is (monitor_emotion, visits etc.). We don't create them here.
    conn.commit()
    cur.close()

# === Helper Functions ===

In [6]:
def l2_normalize(vec):
    vec = np.array(vec, dtype=np.float64)
    norm = np.linalg.norm(vec)
    if norm == 0:
        return vec
    return vec / norm

# Generate timestamped unique face id
# def generate_new_face_id():
#     timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
#     rand = str(random.randint(1000, 9999))
#     return f"face_{timestamp}_{rand}"

# Blur check
def is_blurry_image(img_path, threshold=BLUR_THRESHOLD):
    img = cv2.imread(img_path)
    if img is None:
        return True
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    fm = cv2.Laplacian(gray, cv2.CV_64F).var()
    return fm < threshold

def process_captured_faces():
    conn = db  # use your existing db connection
    cur = conn.cursor(dictionary=True)
    # Get unprocessed images from DB, FIFO order
    cur.execute("""
        SELECT id, image_path, face_id FROM captured_snapshots
        WHERE processed = 0
        ORDER BY timestamp ASC
    """)
    rows = cur.fetchall()
    for row in rows:
        image_path = row['image_path']
        face_id = row['face_id']
        db_id = row['id']
        if not os.path.exists(image_path):
            print(f"❌ Image not found: {image_path}")
            # Optionally mark as processed or remove from DB
            continue

        # Analyze emotion
        dominant, conf = analyze_emotion_from_path(image_path)
        if dominant and conf is not None and conf >= 50:
            # Update DB with emotion and mark as processed
            cur2 = conn.cursor()
            cur2.execute("""
                UPDATE captured_snapshots
                SET emotion=%s, processed=1
                WHERE id=%s
            """, (dominant, db_id))
            conn.commit()
            cur2.close()
            print(f"✅ Processed {image_path} | Face ID: {face_id} | Emotion: {dominant} ({conf:.2f}%)")
        else:
            print(f"⚠️ Low confidence or no emotion for {image_path} (conf={conf}) - not updating DB")

        # Move the image to the archive folder after processing
        archive_image(image_path)

    cur.close()
    
# DeepFace embedding extraction (single face image path)
def get_embedding_from_path(img_path):
    try:
        # DeepFace.represent returns a list of dicts (one per detected face). We expect single-crop images.
        reps = DeepFace.represent(img_path=img_path, model_name=MODEL_NAME, enforce_detection=False)
        if not reps:
            return None
        emb = reps[0]['embedding']
        return np.array(emb, dtype=np.float64)
    except Exception as e:
        print(f"❌ get_embedding_from_path error: {e}")
        return None

# Extract multiple face crops from an image using DeepFace.extract_faces
# returns list of (crop_image, region dict)
def crop_faces(image_path):
    try:
        detections = DeepFace.extract_faces(img_path=image_path, detector_backend='opencv', enforce_detection=False)
        img = cv2.imread(image_path)
        faces = []
        for det in detections:
            region = det.get('facial_area')
            if not region:
                continue
            x, y, w, h = region['x'], region['y'], region['w'], region['h']
            # clamp coordinates
            x1 = max(0, x)
            y1 = max(0, y)
            x2 = min(img.shape[1], x + w)
            y2 = min(img.shape[0], y + h)
            face_img = img[y1:y2, x1:x2]
            if face_img is None or face_img.size == 0:
                continue
            faces.append((face_img, region))
        return faces
    except Exception as e:
        print(f"❌ crop_faces failed: {e}")
        return []

# Load embeddings from DB into memory as: { face_id: [np.array(...), ...] }
def load_known_embeddings(conn):
    known = {}
    cur = conn.cursor()
    cur.execute("SELECT face_id, embedding FROM face_embeddings ORDER BY created_at ASC")
    rows = cur.fetchall()
    for face_id, emb_json in rows:
        try:
            emb = np.array(json.loads(emb_json), dtype=np.float64)
            emb = l2_normalize(emb)
            known.setdefault(face_id, []).append(emb)
        except Exception:
            continue
    cur.close()
    return known

# Save embedding to DB (face_embeddings) and also ensure unique_face_id updated once
def save_embedding_to_db(conn, face_id, embedding):
    cur = conn.cursor()
    now = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    emb_json = json.dumps(embedding.tolist())
    try:
        cur.execute("INSERT INTO face_embeddings (face_id, embedding, created_at) VALUES (%s, %s, %s)", (face_id, emb_json, now))
        # If unique_face_id doesn't have an entry, insert the first embedding for compatibility
        cur.execute("SELECT face_id FROM unique_face_id WHERE face_id = %s", (face_id,))
        if not cur.fetchone():
            cur.execute("INSERT INTO unique_face_id (face_id, embedding, created_at) VALUES (%s, %s, %s)", (face_id, emb_json, now))
        conn.commit()
    except mysql.connector.Error as err:
        print(f"❌ MySQL save_embedding_to_db error: {err}")
        conn.rollback()
    finally:
        cur.close()

# Match embedding against known set. Returns (best_face_id, best_distance) or (None, None)
def match_embedding(embedding, known_embeddings, threshold=MATCH_THRESHOLD):
    best_id = None
    best_dist = float('inf')

    for face_id, emb_list in known_embeddings.items():
        for known_emb in emb_list:
            # embeddings assumed normalized
            dist = cosine(embedding, known_emb)
            if dist < best_dist:
                best_dist = dist
                best_id = face_id

    if best_id is not None and best_dist <= threshold:
        return best_id, best_dist
    return None, None

# Emotion analysis and logging functions (assumes monitor_emotion and emotions tables exist)
def insert_emotion(conn, face_id, emotion, confidence):
    try:
        cur = conn.cursor()
        ts = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        cur.execute("INSERT INTO monitor_emotion (face_id, detected_emotion, confidence, timestamp) VALUES (%s,%s,%s,%s)",
                    (face_id, emotion, float(confidence), ts))
        cur.execute("INSERT INTO emotions (face_id, detected_emotion, confidence, timestamp) VALUES (%s,%s,%s,%s)",
                    (face_id, emotion, float(confidence), ts))
        conn.commit()
        cur.close()
        print(f"📊 Emotion logged for {face_id}: {emotion} ({confidence:.2f}%)")
    except Exception as e:
        print(f"❌ insert_emotion error: {e}")
        try:
            conn.rollback()
        except Exception:
            pass

# Visit logging (simple 1-minute debounce) - assumes visits and monitor_visit / visit_details tables exist
def should_log_visit(conn, face_id, min_seconds=60):
    try:
        cur = conn.cursor()
        cur.execute("SELECT visit_time FROM visits WHERE user_id = %s ORDER BY visit_time DESC LIMIT 1", (face_id,))
        row = cur.fetchone()
        cur.close()
        if not row:
            return True
        last = row[0]
        diff = (datetime.now() - last).total_seconds()
        return diff >= min_seconds
    except Exception:
        return True

# archive image
def archive_image(src_path):
    try:
        ts = datetime.now()
        year = str(ts.year)
        month = ts.strftime('%B')
        archive_dir = os.path.join(ARCHIVE_ROOT, year, month)
        os.makedirs(archive_dir, exist_ok=True)
        dest = os.path.join(archive_dir, os.path.basename(src_path))
        shutil.move(src_path, dest)
        print(f"📁 Archived {os.path.basename(src_path)} -> {archive_dir}")
        return dest
    except Exception as e:
        print(f"❌ archive_image error: {e}")
        return None

# analyze emotion via DeepFace.analyze (single-crop image)
def analyze_emotion_from_path(img_path):
    try:
        res = DeepFace.analyze(img_path=img_path, actions=['emotion'], enforce_detection=False)
        if isinstance(res, list):
            res = res[0]
        dominant = res.get('dominant_emotion')
        confidence = None
        emotions = res.get('emotion')
        if dominant and emotions and dominant in emotions:
            confidence = emotions[dominant]
        return dominant, confidence
    except Exception as e:
        print(f"⚠️ analyze_emotion_from_path error: {e}")
        return None, None

# === Monitoring Logic ===

In [7]:
def get_db_connection():
    import mysql.connector
    # Update with your actual DB credentials
    return mysql.connector.connect(
        host="localhost",
        user="root",
        password="",
        database="emotion_detection"
    )

def monitor_folder():
    os.makedirs(WATCH_FOLDER, exist_ok=True)
    os.makedirs(KNOWN_FOLDER, exist_ok=True)

    conn = get_db_connection()
    ensure_tables_exist(conn)

    print(f"📁 Monitoring {WATCH_FOLDER} ...")

    # Load embeddings into memory
    known_embeddings = load_known_embeddings(conn)
    print(f"🔁 Loaded embeddings for {len(known_embeddings)} known faces")

    processed = set()

    try:
        while True:
            files = [f for f in os.listdir(WATCH_FOLDER) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
            for file in files:
                if file in processed:
                    continue
                path = os.path.join(WATCH_FOLDER, file)
                print(f"\n🖼️ New image: {file}")

                faces = crop_faces(path)
                if not faces:
                    print("❌ No faces detected - archiving anyway")
                    archive_image(path)
                    processed.add(file)
                    continue

                # process each face crop independently                
                for i, (face_img, region) in enumerate(faces):
                    temp_name = f"temp_{uuid.uuid4().hex[:8]}.jpg"
                    cv2.imwrite(temp_name, face_img)

                    emb = get_embedding_from_path(temp_name)
                    if emb is None:
                        print("⚠️ Could not extract embedding for face, skipping")
                        os.remove(temp_name)
                        continue

                    emb = l2_normalize(emb)

                    matched_id, dist = match_embedding(emb, known_embeddings)

                    if matched_id:
                        face_id = matched_id
                        print(f"✅ Face matched -> {face_id} (dist={dist:.4f})")
                        # Optionally store this embedding to that face_id to improve robustness
                        try:
                            save_embedding_to_db(conn, face_id, emb)
                            known_embeddings.setdefault(face_id, []).append(emb)
                        except Exception:
                            pass
                    else:
                        print("🆕 New face detected but face_id assignment should be handled at capture stage. Skipping new face_id generation.")
                        # Optionally: skip saving or handle as needed for your workflow

                    # Emotion analysis
                    dominant, conf = analyze_emotion_from_path(temp_name)
                    if dominant and conf is not None and conf >= 50:
                        insert_emotion(conn, face_id, dominant, conf)
                    else:
                        print(f"⚠️ Emotion low-confidence or missing ({conf}) - skipped logging")

                    # Visit logging (very simple)
                    try:
                        if should_log_visit(conn, face_id):
                            cur = conn.cursor()
                            ts = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                            cur.execute("INSERT INTO visits (user_id, emotion, visit_time) VALUES (%s,%s,%s)", (face_id, dominant or 'unknown', ts))
                            cur.execute("INSERT INTO monitor_visit (face_id, visit_time) VALUES (%s,%s)", (face_id, ts))
                            conn.commit()
                            cur.close()
                            print(f"📝 Logged visit for {face_id}")
                        else:
                            print("⏳ Skipped visit logging (debounce)")
                    except Exception as e:
                        print(f"❌ visit logging error: {e}")

                    # cleanup temp
                    os.remove(temp_name)

                # archive the original image after processing all faces
                archive_image(path)

                processed.add(file)
                

            time.sleep(2)
    except KeyboardInterrupt:
        print("⏹️ Stopping monitor (KeyboardInterrupt)")
    finally:
        try:
            conn.close()
        except Exception:
            pass

# === Entry Point ===

In [8]:
if __name__ == '__main__':
    process_captured_faces()

⚠️ Low confidence or no emotion for captured_faces\01K362ZWTA8RSRNX0EFJM1C5SW_20250821_103231.jpg (conf=38.450042724609375) - not updating DB
📁 Archived 01K362ZWTA8RSRNX0EFJM1C5SW_20250821_103231.jpg -> Process\2025\August
✅ Processed captured_faces\01K362ZWTA8RSRNX0EFJM1C5SW_20250821_103247.jpg | Face ID: 01K362ZWTA8RSRNX0EFJM1C5SW | Emotion: angry (68.59%)
📁 Archived 01K362ZWTA8RSRNX0EFJM1C5SW_20250821_103247.jpg -> Process\2025\August
⚠️ Low confidence or no emotion for captured_faces\01K362ZWTA8RSRNX0EFJM1C5SW_20250821_103249.jpg (conf=42.13692855834961) - not updating DB
📁 Archived 01K362ZWTA8RSRNX0EFJM1C5SW_20250821_103249.jpg -> Process\2025\August
✅ Processed captured_faces\01K362ZWTA8RSRNX0EFJM1C5SW_20250821_103250.jpg | Face ID: 01K362ZWTA8RSRNX0EFJM1C5SW | Emotion: angry (60.66%)
📁 Archived 01K362ZWTA8RSRNX0EFJM1C5SW_20250821_103250.jpg -> Process\2025\August
✅ Processed captured_faces\01K362ZWTA8RSRNX0EFJM1C5SW_20250821_103253.jpg | Face ID: 01K362ZWTA8RSRNX0EFJM1C5SW | Emo